<a href="https://colab.research.google.com/github/Pala63/Python-Web-Scraping/blob/main/lessons/02_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping with Beautiful Soup

* * *

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

In [252]:
%pip install requests

In [253]:
%pip install beautifulsoup4

We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [254]:
%pip install lxml

In [255]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracting and Parsing HTML

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

In [256]:
# Make a GET request
req = requests.get('http://www.ilga.gov/Senate/Members')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName" content="Legislative Information System">
    <meta name="contactOrganization" content="LIS Staff Services">
    <meta name="contactStreetAddress1" content="705 Stratton Office Building">
    <meta name="contactCity" content="Springfield">
    <meta name="contactZipcode" content="62706">
    <meta name="contactNetworkAddress" content="webmaster@ilga.gov">
    <meta name="contactPhoneNumber" content="217-782-3944">
    <meta name="contactFaxNumber" content="217-524-6059">
    <meta name


En esta celda se hace una solicitud HTTP (GET) a la página del Senado del estado de Illinois usando la librería requests.
Luego se guarda el contenido HTML de la página en la variable src.
Finalmente, se imprimen los primeros 1000 caracteres del HTML para ver cómo está estructurado el código de esa página web.

¿Por qué hacemos esto?
Para hacer scraping, necesitamos primero ver cómo está estructurado el HTML, ya que es ahí donde está la información que queremos extraer. En este caso, la página del Senado tiene los nombres y enlaces a los senadores, y lo vamos a capturar desde el HTML directamente.

## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

In [257]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" name="contactName"/>
  <meta content="LIS Staff Services" name="contactOrganization"/>
  <meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
  <meta content="Springfield" name="contactCity"/>
  <meta content="62706" name="contactZipcode"/>
  <meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
  <meta content="217-782-3944" name="contactPhoneNumber"/>
  <meta content="217-524-6059" name="contactFaxNumber"/>
  <meta content="State Of Illinois" name="originatorJur

Se convierte el contenido HTML (src) en un objeto BeautifulSoup.
Esto permite navegar y buscar información dentro del HTML como si fuera un árbol estructurado.
Se usa 'lxml' como el parser porque es rápido y eficiente.
La función prettify() muestra el HTML con sangrías, facilitando su lectura.
Aquí solo se imprimen los primeros 1000 caracteres para no saturar la salida

The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**.

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

In [258]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="af" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-za"></span> Afrikaans
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="sq" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-al"></span> Albanian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="ar" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-ae"></span> Arabic
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="hy" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-am"></span> Armenian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="az" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-az"></span> Azerbaijani
            

Se buscan todas las etiquetas <a> en la página (que normalmente representan enlaces).
find_all("a") devuelve una lista con todos los elementos <a>.
Se imprimen los primeros 10 resultados para ver cómo lucen.
Esto es útil para ver a qué páginas están enlazando (como los senadores en este caso).

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object.

These two lines of code are equivalent:

In [259]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>


Estas dos formas (soup.find_all("a") y soup("a")) hacen exactamente lo mismo.
Ambas devuelven una lista con todos los elementos <a> del HTML.
Se imprimen los primeros elementos de ambas listas para mostrar que son iguales.

How many links did we obtain?

In [260]:
print(len(a_tags))

270


Se imprime la cantidad total de etiquetas <a> encontradas en la página.
Generalmente habrá muchos enlaces, y no todos serán útiles para tu objetivo.

That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes?

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

In [261]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="notranslate")
side_menus[:2]

[<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>]

Se filtran los enlaces <a> que tienen el atributo class="sidemenu".
Esto es útil para obtener solo los enlaces que forman parte del menú lateral de navegación

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [262]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("div.member-overlay")
selected[:5]

[<div class="member-overlay">
 <h5 class="card-title"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a> (R)</h5>
 <p class="card-text">
                                             Republican Caucus Chair
                                             <br/>47th District
                                         </p>
 </div>,
 <div class="member-overlay">
 <h5 class="card-title"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a> (R)</h5>
 <p class="card-text">
                                             Republican Caucus Chair
                                             <br/>47th District
                                         </p>
 </div>,
 <div class="member-overlay">
 <h5 class="card-title"><a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a> (D)</h5>
 <p class="card-text">
                                             Majority Caucus Chair
                                             <br/>2nd District
       

Otra forma más eficiente de buscar elementos es con selectores CSS.
"a.sidemenu" selecciona todas las etiquetas <a> que tienen la clase sidemenu.
Se imprimen los primeros 5 elementos encontrados.
Este método es muy flexible y permite búsquedas complejas como "div > ul > li > a".

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `mainmenu`.

In [263]:
# YOUR CODE HERE
side_menus = soup.find_all("a", href="/Legislation")
side_menus[:10]


[<a aria-expanded="false" aria-haspopup="true" b-0yw6sxot5c="" data-toggle="dropdown" href="/Legislation" role="button">
 <span b-0yw6sxot5c="">LEGISLATION &amp; LAWS</span> <i b-0yw6sxot5c="" class="fa fa-chevron-down"></i>
 </a>,
 <a b-0yw6sxot5c="" href="/Legislation">Bills &amp; Resolutions</a>]

## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

In [264]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.notranslate")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>
Class:  <class 'bs4.element.Tag'>


It's a Beautiful Soup tag! This means it has a `text` member:

In [265]:
print(first_link.text)

Neil Anderson


Esto es interesante por que ignora las etiquetas html y me devuelve solo el texto.

Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

In [266]:
print(first_link['href'])

/Senate/Members/Details/3312


## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

In [267]:
print(soup.prettify()[:500])  # Solo los primeros 500 caracteres para no saturar


<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" nam


In [268]:
# YOUR CODE HERE
# Buscar todos los enlaces con clase 'dropdown-item'
no_translate = soup.find_all('a', class_='notranslate')

# Extraer sus atributos href
dropdown_hrefs = [link['href'] for link in no_translate]

# Mostrar resultados
print(dropdown_hrefs)


['/Senate/Members/Details/3312', '/Senate/Members/Details/3312', '/Senate/Members/Details/3316', '/Senate/Members/Details/3316', '/Senate/Members/Details/3383', '/Senate/Members/Details/3383', '/Senate/Members/Details/3413', '/Senate/Members/Details/3413', '/Senate/Members/Details/3337', '/Senate/Members/Details/3337', '/Senate/Members/Details/3386', '/Senate/Members/Details/3386', '/Senate/Members/Details/3317', '/Senate/Members/Details/3317', '/Senate/Members/Details/3403', '/Senate/Members/Details/3403', '/Senate/Members/Details/3410', '/Senate/Members/Details/3410', '/Senate/Members/Details/3443', '/Senate/Members/Details/3443', '/Senate/Members/Details/3291', '/Senate/Members/Details/3291', '/Senate/Members/Details/3329', '/Senate/Members/Details/3329', '/Senate/Members/Details/3334', '/Senate/Members/Details/3334', '/Senate/Members/Details/3407', '/Senate/Members/Details/3407', '/Senate/Members/Details/3339', '/Senate/Members/Details/3339', '/Senate/Members/Details/3412', '/Senat

<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [269]:
# Make a GET request
req = requests.get('https://ilga.gov/Legislation/BillStatus?DocNum=818&DocTypeID=HR&GA=101&GAID=9&LegID=34456&SessionID=51')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

**WEB DINAMICA**

Al realizar un scraping de la página https://ilga.gov/Legislation/BillStatus?... utilizando requests y BeautifulSoup, me encontré con un problema: la búsqueda de elementos <tr> (filas de tabla) devolvía una lista vacía. Esto se debe a que la página carga su contenido de forma dinámica utilizando JavaScript.

Cuando utilizamos requests.get(...), lo que se descarga es únicamente el HTML estático que el servidor entrega inicialmente. Sin embargo, en este caso, la tabla que contiene los datos no se encuentra en ese HTML original, sino que es generada posteriormente por JavaScript una vez que la página ha sido completamente cargada en el navegador. Como requests no tiene la capacidad de ejecutar código JavaScript, dicha tabla simplemente no aparece en el contenido descargado.

Por esta razón, decidí utilizar Selenium, una herramienta que permite simular el comportamiento de un navegador real (como Chrome o Firefox). Selenium sí ejecuta JavaScript, lo que permite acceder al contenido completo de la página, incluyendo los datos generados dinámicamente. Gracias a esto, fue posible capturar el HTML final ya renderizado y, a partir de él, utilizar BeautifulSoup para encontrar correctamente todas las filas <tr> que componen la tabla deseada.

In [270]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)
print("¿Hay '<tr>' en el HTML descargado?", "<tr" in src)
print("Primeros 500 caracteres:\n", src[:500])


¿Hay '<tr>' en el HTML descargado? False
Primeros 500 caracteres:
 <!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName


In [271]:
pip install selenium


In [272]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # Import Options
from bs4 import BeautifulSoup
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') # Add this argument
chrome_options.add_argument('--headless') # Add this argument for running without a visible browser
chrome_options.add_argument('--disable-dev-shm-usage') # Add this argument

# Crear el navegador
driver = webdriver.Chrome(options=chrome_options) # Pass the options

try:
    # Cargar la página
    driver.get('https://ilga.gov/Legislation/BillStatus?DocNum=818&DocTypeID=HR&GA=101&GAID=9&LegID=34456&SessionID=51')

    # Esperar unos segundos para que cargue el contenido dinámico
    time.sleep(5)

    # Obtener el HTML ya renderizado con JavaScript
    html = driver.page_source

    # Analizar con BeautifulSoup
    soup = BeautifulSoup(html, 'lxml')

    # Buscar filas de tabla
    rows = soup.find_all("tr")
    print(f"Número de filas encontradas: {len(rows)}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit() # Ensure the driver is closed

Número de filas encontradas: 18


⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [273]:
# Returns every ‘tr tr tr’ css selector in the page
# rows = soup.select('tr tr tr')

# for row in rows[:5]:
    #print(row, '\n')

Este codigo no encuentra etiquetas anidadas dentro de otras tr tres niveles profundos.

In [274]:
rows = soup.select('table tr')  # Todas las filas dentro de cualquier tabla
print(f"Número de filas encontradas: {len(rows)}")

if rows:
    for row in rows[:5]:
        print(row.prettify(), '\n')
else:
    print("No se encontraron filas <tr> en la página.")


Número de filas encontradas: 18
<tr>
 <th>
  Date
 </th>
 <th>
  Chamber
 </th>
 <th>
  Action
 </th>
</tr>
 

<tr>
 <td align="left" class="content" valign="top" width="13%">
  11/02/2007
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" width="75%">
  Filed with the Clerk by
  <a href="../../house/members/details/1307">
   Rep. Daniel V. Beiser
  </a>
 </td>
</tr>
 

<tr>
 <td align="left" class="content" valign="top" width="13%">
  11/07/2007
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" width="75%">
  Added Chief Co-Sponsor
  <a href="../../house/members/details/1291">
   Rep. John E. Bradley
  </a>
 </td>
</tr>
 

<tr>
 <td align="left" class="content" valign="top" width="13%">
  11/07/2007
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" widt

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [275]:
example_row = rows[2]
print(example_row.prettify())

<tr>
 <td align="left" class="content" valign="top" width="13%">
  11/07/2007
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" width="75%">
  Added Chief Co-Sponsor
  <a href="../../house/members/details/1291">
   Rep. John E. Bradley
  </a>
 </td>
</tr>



Este codigo imprime 3 filas 0 1 2

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

In [276]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.content'):
    print(cell)
print()

for cell in example_row.select('td.content'):
    print(cell)
print()

<td align="left" class="content" valign="top" width="13%">
11/07/2007                                        </td>
<td align="left" class="content" valign="top" width="12%">
House                                        </td>
<td align="left" class="content" valign="top" width="75%">
Added Chief Co-Sponsor <a href="../../house/members/details/1291">Rep. John E. Bradley</a></td>

<td align="left" class="content" valign="top" width="13%">
11/07/2007                                        </td>
<td align="left" class="content" valign="top" width="12%">
House                                        </td>
<td align="left" class="content" valign="top" width="75%">
Added Chief Co-Sponsor <a href="../../house/members/details/1291">Rep. John E. Bradley</a></td>

<td align="left" class="content" valign="top" width="13%">
11/07/2007                                        </td>
<td align="left" class="content" valign="top" width="12%">
House                                        </td>
<td align="le

We can confirm that these are all the same.

In [277]:
assert example_row.select('td') == example_row.select('.content') == example_row.select('td.content')

Let's use the selector `td.detail` to be as specific as possible.

In [278]:
# Select only those 'td' tags with class 'detail'
detail_cells = example_row.select('td.content')
detail_cells

[<td align="left" class="content" valign="top" width="13%">
 11/07/2007                                        </td>,
 <td align="left" class="content" valign="top" width="12%">
 House                                        </td>,
 <td align="left" class="content" valign="top" width="75%">
 Added Chief Co-Sponsor <a href="../../house/members/details/1291">Rep. John E. Bradley</a></td>]

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [279]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

['\n11/07/2007                                        ', '\nHouse                                        ', '\nAdded Chief Co-Sponsor Rep. John E. Bradley']


Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

In [280]:
print(row_data[0]) # Name
print(row_data[1]) # District
print(row_data[2]) # Party


11/07/2007                                        

House                                        

Added Chief Co-Sponsor Rep. John E. Bradley


## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [281]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <tr>
<th>Date</th>
<th>Chamber</th>
<th>Action</th>
</tr> 

Row 1:
 <tr>
<td align="left" class="content" valign="top" width="13%">
11/02/2007                                        </td>
<td align="left" class="content" valign="top" width="12%">
House                                        </td>
<td align="left" class="content" valign="top" width="75%">
Filed with the Clerk by <a href="../../house/members/details/1307">Rep. Daniel V. Beiser</a></td></tr> 

Last Row:
 <tr>
<td align="left" class="content" valign="top" width="13%">
<b>5/19/2008</b>
</td>
<td align="left" class="content" valign="top" width="12%">
<b>House</b>
</td>
<td align="left" class="content" valign="top" width="75%">
<b>Tabled By Sponsor <a href="../../house/members/details/1307">Rep. Daniel V. Beiser</a></b>
</td></tr>


When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

In [282]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

7
6
6
6


Perhaps good rows have a length of 5. Let's check:

In [283]:
# Filtrar filas con 3 celdas <td>
good_rows = [row for row in rows if len(row.find_all('td')) == 3]

print(f"Filas con 3 celdas encontradas: {len(good_rows)}\n")

# Mostrar la primera fila bien formada
print(good_rows[0].prettify(), '\n')

# Mostrar la penúltima fila
print(good_rows[-2].prettify(), '\n')

# Mostrar la última fila
print(good_rows[-1].prettify())


Filas con 3 celdas encontradas: 17

<tr>
 <td align="left" class="content" valign="top" width="13%">
  11/02/2007
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" width="75%">
  Filed with the Clerk by
  <a href="../../house/members/details/1307">
   Rep. Daniel V. Beiser
  </a>
 </td>
</tr>
 

<tr>
 <td align="left" class="content" valign="top" width="13%">
  5/19/2008
 </td>
 <td align="left" class="content" valign="top" width="12%">
  House
 </td>
 <td align="left" class="content" valign="top" width="75%">
  Motion Prevailed
 </td>
</tr>
 

<tr>
 <td align="left" class="content" valign="top" width="13%">
  <b>
   5/19/2008
  </b>
 </td>
 <td align="left" class="content" valign="top" width="12%">
  <b>
   House
  </b>
 </td>
 <td align="left" class="content" valign="top" width="75%">
  <b>
   Tabled By Sponsor
   <a href="../../house/members/details/1307">
    Rep. Daniel V. Beiser
   </a>
  </b>
 </td>
<

We found a footer row in our list that we'd like to avoid. Let's try something else:

In [284]:
rows[2].select('td.content')

[<td align="left" class="content" valign="top" width="13%">
 11/07/2007                                        </td>,
 <td align="left" class="content" valign="top" width="12%">
 House                                        </td>,
 <td align="left" class="content" valign="top" width="75%">
 Added Chief Co-Sponsor <a href="../../house/members/details/1291">Rep. John E. Bradley</a></td>]

In [285]:
# Bad row
print(rows[-1].select('td.content'), '\n')

# Good row
print(rows[5].select('td.content'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.content')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[<td align="left" class="content" valign="top" width="13%">
<b>5/19/2008</b>
</td>, <td align="left" class="content" valign="top" width="12%">
<b>House</b>
</td>, <td align="left" class="content" valign="top" width="75%">
<b>Tabled By Sponsor <a href="../../house/members/details/1307">Rep. Daniel V. Beiser</a></b>
</td>] 

[<td align="left" class="content" valign="top" width="13%">
11/07/2007                                        </td>, <td align="left" class="content" valign="top" width="12%">
House                                        </td>, <td align="left" class="content" valign="top" width="75%">
Added Chief Co-Sponsor <a href="../../house/members/details/1348">Rep. Fred Crespo</a></td>] 

Checking rows...

<tr>
<td align="left" class="content" valign="top" width="13%">
11/02/2007                                        </td>
<td align="left" class="content" valign="top" width="12%">
House                                        </td>
<td align="left" class="content" valign="top"

Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

In [286]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.content')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.content')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    Date = row_data[0]
    Chamber = row_data[1]
    Action = row_data[2]
    # Store in a tuple
    senator = (Date, Chamber, Action)
    # Append to list
    members.append(senator)

In [287]:
# Should be 61
len(members)

17

Let's take a look at what we have in `members`.

In [288]:
print(members[:5])

[('\n11/02/2007                                        ', '\nHouse                                        ', '\nFiled with the Clerk by Rep. Daniel V. Beiser'), ('\n11/07/2007                                        ', '\nHouse                                        ', '\nAdded Chief Co-Sponsor Rep. John E. Bradley'), ('\n11/07/2007                                        ', '\nHouse                                        ', '\nAdded Chief Co-Sponsor Rep. Robert F. Flider'), ('\n11/07/2007                                        ', '\nHouse                                        ', '\nAdded Chief Co-Sponsor Rep. Elizabeth Hernandez'), ('\n11/07/2007                                        ', '\nHouse                                        ', '\nAdded Chief Co-Sponsor Rep. Fred Crespo')]


## 🥊  Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format.

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`.

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips:

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [289]:
# # Make a GET request
# req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# # Read the content of the server’s response
# src = req.text
# # Soup it
# soup = BeautifulSoup(src, "lxml")
# # Create empty list to store our data
# members = []

# # Returns every ‘tr tr tr’ css selector in the page
# rows = soup.select('tr tr tr')
# # Get rid of junk rows
# rows = [row for row in rows if row.select('td.detail')]

# # Loop through all rows
# for row in rows:
#     # Select only those 'td' tags with class 'detail'
#     detail_cells = row.select('td.detail')
#     # Keep only the text in each of those cells
#     row_data = [cell.text for cell in detail_cells]
#     # Collect information
#     name = row_data[0]
#     district = int(row_data[3])
#     party = row_data[4]

#     # YOUR CODE HERE
#     full_path = ''

#     # Store in a tuple
#     senator = (name, district, party, full_path)
#     # Append to list
#     members.append(senator)

In [295]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

# Make a GET request
req = requests.get('https://ilga.gov/Senate/Members/List')
src = req.text

# Soup it
soup = BeautifulSoup(src, "lxml")

# Create empty list to store our data
members = []

# Encuentra todos los bloques de senadores
rows = soup.select('.row.member-entry')

# Loop through todos los bloques
for row in rows:
    # Obtener nombre
    name_tag = row.select_one('.col-md-3 strong')
    name = name_tag.text.strip() if name_tag else None

    # Obtener partido y distrito
    details = row.select_one('.col-md-3').get_text(separator='|', strip=True).split('|')
    party = details[-1] if len(details) >= 2 else None
    district = details[-2] if len(details) >= 2 else None
    try:
        district = int(district.replace('District ', '').strip())
    except:
        district = None

    # Buscar enlace con MemberID
    anchor = row.select_one('a[href*="MemberID"]')
    if anchor and 'href' in anchor.attrs:
        href = anchor['href']
        query = urlparse(href).query
        member_id = parse_qs(query).get('MemberID', [None])[0]

        if member_id:
            full_path = f"https://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID={member_id}&Primary=True"
        else:
            full_path = None
    else:
        full_path = None

    # Guardar tupla
    senator = (name, district, party, full_path)
    members.append(senator)

# Mostrar primeros 5
for m in members[:5]:
    print(m)



In [291]:
# Uncomment to test
# members[:5]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator.

In [292]:
# YOUR CODE HERE
def get_members(url):
    return [___]


In [293]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

1

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [294]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        bill_id =
        description =
        chamber =
        last_action =
        last_action_date =
        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills

SyntaxError: invalid syntax (ipython-input-294-3384773414.py, line 8)

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# YOUR CODE HERE


In [ ]:
# Uncomment to test your code
# bills_dict[52]